# Importaciones

In [ ]:
!pip install gdown --quiet

In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import pickle
from google.colab import files
import gdown
import gradio as gr

In [ ]:
# Descargamos el Dataset desde el enlace
url = "https://drive.google.com/uc?export=download&id=1N9r83W0u8cukYK5VH-_jQTBKp06Xity8"
# Leemos el Dataset
df = pd.read_csv(url)

# Visualización inicial

In [ ]:
# Ver tamaño
df.shape

In [ ]:
# Ver tipos de datos
df.dtypes

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
# Nulos por columna
df.isnull().sum()

In [ ]:
# Distribución de la columna result
df['Injury_Next_Season'].value_counts()

In [ ]:
# Calcular correlación con la variable objetivo
correlaciones = df.corr(numeric_only=True)['Injury_Next_Season'].sort_values(ascending=False)

plt.figure(figsize=(8,6))
sns.barplot(x=correlaciones.values, y=correlaciones.index, palette='coolwarm')
plt.title("Correlación de cada variable con Injury_Next_Season")
plt.xlabel("Coeficiente de correlación (Pearson)")
plt.ylabel("Variable")
plt.show()

In [ ]:
# Calcular correlación absoluta con la variable objetivo
correlaciones = df.corr(numeric_only=True)['Injury_Next_Season'].abs().sort_values(ascending=False)

plt.figure(figsize=(8,6))
sns.barplot(x=correlaciones.values, y=correlaciones.index, palette='viridis')
plt.title("Fuerza de correlación (sin signo) con Injury_Next_Season")
plt.xlabel("|Coeficiente de correlación (Pearson)|")
plt.ylabel("Variable")
plt.show()

Las variables que mas influyen en el resultado final son Stress_Level_Score y Sleep_Hours_Per_Night

In [ ]:
df["Position"].unique()

# Preprocesamiento del Dataframe

In [ ]:
# Definir variables X e y
X = df.drop("Injury_Next_Season", axis=1)
y = df["Injury_Next_Season"]

In [ ]:
print("Tamaño de X:", X.shape)
print("Tamaño de y:", y.shape)
print("Clases en y:", y.unique())

In [ ]:
# Dividimos en Train, Val y Test
# Train: 65%, Validación: 22.8% (~23%), Test: 12.2% (~12%)

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.35, random_state=42, stratify=y
    )

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.35, random_state=42, stratify=y_temp
    )

In [ ]:
print("Train:", X_train.shape[0], "filas")
print("Validación:", X_val.shape[0], "filas")
print("Test:", X_test.shape[0], "filas")

In [ ]:
# Identificar columnas numéricas y categóricas
categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(include=['int64','float64']).columns

print("Categóricas:", categorical_cols.tolist())
print("Numéricas:", numeric_cols.tolist())

In [ ]:
# Inicializar el encoder
ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)

In [ ]:
# Transformar columnas categóricas a numéricas

# Ajustar y transformar solo en train
X_train_cat = ohe.fit_transform(X_train[categorical_cols])

# Transformar val y test con el mismo encoder
X_val_cat = ohe.transform(X_val[categorical_cols])
X_test_cat = ohe.transform(X_test[categorical_cols])

In [ ]:
# Combinar con columnas numéricas
X_train = np.hstack([X_train[numeric_cols].values, X_train_cat])
X_val = np.hstack([X_val[numeric_cols].values, X_val_cat])
X_test = np.hstack([X_test[numeric_cols].values, X_test_cat])

print("Shape X_train:", X_train.shape)

In [ ]:
# Inicializar el escalador
scaler = StandardScaler()

In [ ]:
# Escalamos solo con los datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Transformamos validación y test con el mismo escalador
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Definir el modelo

In [ ]:
# Definir SVM con kernel RBF
svm_model = SVC(
    kernel='rbf',
    C=1.0,          # penalización por error
    gamma='scale',  # parámetro del kernel RBF
    random_state=42
)

In [ ]:
# Entrenar el modelo con el conjunto de entrenamiento
svm_model.fit(X_train_scaled, y_train)

# Validar el modelo en el conjunto de validación

In [ ]:
# Predecir en validación
y_val_pred = svm_model.predict(X_val_scaled)

In [ ]:
# Evaluar
print("Accuracy en validación:", accuracy_score(y_val, y_val_pred))

In [ ]:
print("\nMatriz de confusión:\n", confusion_matrix(y_val, y_val_pred))

In [ ]:
print("\nReporte de clasificación:\n", classification_report(y_val, y_val_pred))

# Mejora de hiperparámetros

In [ ]:
# Definir el nuevo modelo
svm_model_mej = SVC(random_state=42)

In [ ]:
# Definir los parámetros a probar
param_grid = {
    'C': [0.1, 0.5, 1.0, 5.0],           # Penalización por error
    'gamma': ['scale', 0.1, 0.5, 1.0],    # Influencia de cada punto
    'kernel': ['rbf', 'linear', 'poly']   # Tipos de kernel
}

In [ ]:
# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_model_mej,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,                 # validación cruzada 5 folds
    verbose=2,
    n_jobs=-1             # usa todos los núcleos de tu CPU para acelerar el cálculo
)

La validación cruzada (cross-validation) es una técnica para evaluar cómo de bien generaliza el modelo a datos nuevos, en lugar de usar una sola partición train/test, divide los datos de entrenamiento en varias partes (llamadas folds), y prueba el modelo varias veces.

En este caso como hemos puesto fold 5, los datos se dividiran en 5 partes, el modelo se ejecutará 5 veces, en cada ejecución el modelo se entrenará con 4 folds y se validará con el fold restante, cambiando de fold de validación en cada ejecución

In [ ]:
grid_search.fit(X_train_scaled, y_train)

In [ ]:
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor score en CV:", grid_search.best_score_)

In [ ]:
final_svm_model = grid_search.best_estimator_

In [ ]:
# Predecir en validación
y_val_pred_final = final_svm_model.predict(X_val_scaled)

In [ ]:
# Evaluar
print("Accuracy en validación:", accuracy_score(y_val, y_val_pred_final))

# Evaluar el rendimiento en el conjunto de test

In [ ]:
y_test_pred = final_svm_model.predict(X_test_scaled)

In [ ]:
accuracy = accuracy_score(y_test, y_test_pred)
print("Accuracy en test:", accuracy)

In [ ]:
# Reporte de clasificación (precision, recall, f1-score)
print("\nReporte de clasificación:\n", classification_report(y_test, y_test_pred))

In [ ]:
# Matríz de confusión
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)

plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - Test')
plt.show()

# Serialización del modelo

In [ ]:
# Diccionario con todos los objetos
objetos_a_guardar = {
    "modelo": final_svm_model,    # SVM entrenado
    "encoder": ohe,        # OneHotEncoder
    "scaler": scaler       # StandardScaler
}

In [ ]:
# Guardar en Colab
archivo_pickle = "svm_model.pkl"
with open(archivo_pickle, "wb") as f:
    pickle.dump(objetos_a_guardar, f)

# Descargar automáticamente
files.download(archivo_pickle)

print("Modelo y preprocesadores guardados correctamente")

# Cargar modelo .pkl

In [ ]:
url = "https://drive.google.com/uc?export=download&id=1gBbozuuzCA81QbieOlYazhEOESnrFoRw"
output = "svm_model_lesiones.pkl"

# Descargar desde Drive
gdown.download(url, output, quiet=False)

# Cargar el modelo y preprocesadores
with open(output, "rb") as f:
    data_cargada = pickle.load(f)

modelo_cargado = data_cargada["modelo"]
ohe_cargado = data_cargada["encoder"]
scaler_cargado = data_cargada["scaler"]

print("\nModelo y preprocesadores cargados correctamente desde Google Drive.")

In [ ]:
# Probar modelo cargado en el conjunto de test
y_test_pkl = modelo_cargado.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_test_pkl)
print("Accuracy en test:", accuracy)

# Gradio

In [ ]:
categorical_cols = ["Position"]

numeric_cols = [
    "Age", "Height_cm", "Weight_kg", "Training_Hours_Per_Week",
    "Matches_Played_Past_Season", "Previous_Injury_Count", "Knee_Strength_Score",
    "Hamstring_Flexibility", "Reaction_Time_ms", "Balance_Test_Score",
    "Sprint_Speed_10m_s", "Agility_Score", "Sleep_Hours_Per_Night",
    "Stress_Level_Score", "Nutrition_Quality_Score",
    "Warmup_Routine_Adherence", "BMI"
]

In [ ]:
def predecir_lesion(
    Position,
    Age, Height_cm, Weight_kg, Training_Hours_Per_Week,
    Matches_Played_Past_Season, Previous_Injury_Count, Knee_Strength_Score,
    Hamstring_Flexibility, Reaction_Time_ms, Balance_Test_Score,
    Sprint_Speed_10m_s, Agility_Score, Sleep_Hours_Per_Night,
    Stress_Level_Score, Nutrition_Quality_Score, Warmup_Routine_Adherence, BMI
):
    # Construir arrays
    X_num = np.array([[Age, Height_cm, Weight_kg, Training_Hours_Per_Week,
                       Matches_Played_Past_Season, Previous_Injury_Count, Knee_Strength_Score,
                       Hamstring_Flexibility, Reaction_Time_ms, Balance_Test_Score,
                       Sprint_Speed_10m_s, Agility_Score, Sleep_Hours_Per_Night,
                       Stress_Level_Score, Nutrition_Quality_Score,
                       Warmup_Routine_Adherence, BMI]])

    X_cat = np.array([[Position]])

    # Transformar categóricas y escalar
    X_cat_enc = ohe_cargado.transform(X_cat)
    X_final = np.hstack([X_num, X_cat_enc])
    X_scaled = scaler_cargado.transform(X_final)

    # Predicción
    pred = modelo_cargado.predict(X_scaled)[0]

    # Resultado legible
    if pred == 1:
        return "Lesión probable la próxima temporada"
    else:
        return "Sin riesgo significativo de lesión"

In [ ]:
demo = gr.Interface(
    fn=predecir_lesion,
    inputs=[
        gr.Dropdown(["Defender", "Midfielder", "Forward", "Goalkeeper"], label="Position"),
        gr.Number(label="Age", minimum=0, maximum=1000),
        gr.Number(label="Height (cm)", minimum=0, maximum=1000),
        gr.Number(label="Weight (kg)", minimum=0, maximum=1000),
        gr.Number(label="Training Hours Per Week", minimum=0, maximum=168),
        gr.Number(label="Matches Played Past Season", minimum=0, maximum=1000),
        gr.Number(label="Previous Injury Count", minimum=0, maximum=1000),
        gr.Number(label="Knee Strength Score", minimum=0, maximum=100),
        gr.Number(label="Hamstring Flexibility", minimum=0, maximum=100),
        gr.Number(label="Reaction Time (ms)", minimum=0, maximum=1000),
        gr.Number(label="Balance Test Score", minimum=0, maximum=100),
        gr.Number(label="Sprint Speed 10m (s)", minimum=0, maximum=1000),
        gr.Number(label="Agility Score", minimum=0, maximum=100),
        gr.Number(label="Sleep Hours Per Night", minimum=0, maximum=24),
        gr.Number(label="Stress Level Score", minimum=0, maximum=100),
        gr.Number(label="Nutrition Quality Score", minimum=0, maximum=100),
        gr.Number(label="Warmup Routine Adherence", minimum=0, maximum=1),
        gr.Number(label="BMI", minimum=0, maximum=100),
    ],
    outputs=gr.Textbox(label="Predicción del modelo"),
    title="Predicción de Lesiones Deportivas (SVM)",
    description="Introduce los datos del jugador y el modelo estimará si habrá lesión en la próxima temporada."
)

demo.launch()

# Modelo subido a HuggingFace:


https://huggingface.co/spaces/juanmacbet/lesiones-svm